# Relatively Low Trust in Community

Predicting instances where respondent reported high trust in political and health care but low trust/mistrust of their community. 




Seperate Model for Each Jurisdiction, run automatically

EDA, FE, Cleaning and HP tuning seperate

Dataset stored locally as per terms

In [26]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 
from catboost import *
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
# import shap
from time import time
import plotly.express as px
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from ydata_profiling import ProfileReport
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)

Import Reformatted Dataset, remove trust from X

In [27]:
RawData = pd.read_excel('Reformated.xlsx')

In [31]:
df3 = RawData[['country','age', 'ua', 'os', 'device', 'region', 'gender',
       'wave', 'q00a_select_language', 'q07_live_area',
       'q08_primary_covid_info_source', 'q09_education_completed', 'y5_COMMLwrTrust']]

In [29]:
# df3['country'].unique()

In [32]:
# Store / Initiate 
models = {}
feature_importances = {}
accuracy_scores = {}

COMM_df = None

#Set categorical columns
categorical_features_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# Split the data by country and run a model for each country
for country in df3['country'].unique():
    country_data = df3[df3['country'] == country]
    
   #GROUP RARE INSTANCES 
    counts = country_data['ua'].value_counts()
    infrequent_categories = counts[counts < 100].index   
    country_data['ua'] = country_data['ua'].replace(infrequent_categories, 'other')

    counts = country_data['os'].value_counts()
    infrequent_categories = counts[counts < 100].index   
    country_data['os'] = country_data['os'].replace(infrequent_categories, 'other')

    counts = country_data['device'].value_counts()
    infrequent_categories = counts[counts < 200].index   
    country_data['device'] = country_data['device'].replace(infrequent_categories, 'other')

    counts = country_data['region'].value_counts()
    infrequent_categories = counts[counts < 50].index   
    country_data['region'] = country_data['region'].replace(infrequent_categories, 'other')
    
    # Features and Target
    X = country_data[['age', 'ua', 'os', 'device', 'region', 'gender',
       'wave', 'q00a_select_language', 'q07_live_area',
       'q08_primary_covid_info_source', 'q09_education_completed']]
    
    y = country_data['y5_COMMLwrTrust']
    
        
    #Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25)
    
    # Train a CatBoost classifier
    cbc = CatBoostClassifier(iterations = 500, verbose=0)  
    cbc.fit(X_train, y_train, cat_features= categorical_features_indices)
    
    # Store the model
    models[country] = cbc
    
    # Get feature importance and store in a DataFrame
    importance_df = pd.DataFrame({
        'Feature': X.columns,
        country: cbc.get_feature_importance()
    })
    
    # Merge importance dataframes
    if COMM_df is None:
        COMM_df = importance_df
    else:
        COMM_df = COMM_df.merge(importance_df, on='Feature')
    
    # Predict the Pass column COMM_dffor the test set and compute accuracy
    y_pred = cbc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores[country] = accuracy

In [35]:
# Print out the merged feature importances dataframe
# print(COMM_df)

In [ ]:
# print("\nAccuracy:")
# for country, score in accuracy_scores.items():
#     print(f"{country}: {score:.2f}")

In [36]:
# COMM_df.to_pickle("COMM")